In [ ]:
""" Topic: Time series analysis for predictive maintenance of turbofan engines.
Goal: The project aim is to build a predictive model- used Linear regression - to estimate the Remaining Useful Life ( RUL) of a jet engine based on run-to-failure data of a fleet of similar jet engines.

Data: the datasets consist of 12 files , 4 files for training the data , other 4 files for testig the model , and last 4 for the RUL(Remainging Useful Life) , The FD001 dataset is only considered out of the four datasets that are available. The .txt files that are present under FD001 and are detailedly explained below. train_FD001.txt: Contains 20631 data points from 100 different engines. Each datapoint consists of engine id, cycle (time), 3 operation settings (Operational settings: altitude, throttle resolver angle (TRA), Mach number
) and 21 sensor measurements (consisting of speeds , temperature , and pressures). """

In [ ]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score

In [ ]:
train_df = pd.read_csv('train_FD001.txt', sep=" ", header=None )
test_df = pd.read_csv('test.txt', sep=" ", header=None) 
RUL_df = pd.read_csv('RUL_FD001.txt', sep=" ", header=None) 
train_df.head(5)  

In [ ]:
# Cleaning the data (2 extra columns were added due to white space, deleting those and adding the column names
test_df = test_df[list(range(26))]
train_df = train_df[list(range(26))]
# column names for the dataset
col_list = ['engine_num', 'time', 'op_cond_1', 'op_cond_2', 'op_cond_3', 'sm_1', 'sm_2', 'sm_3', 'sm_4', 'sm_5', 'sm_6', 'sm_7', 'sm_8', 'sm_9', 'sm_10', 'sm_11', 'sm_12', 'sm_13', 'sm_14', 'sm_15', 'sm_16', 'sm_17', 'sm_18', 'sm_19', 'sm_20', 'sm_21']
# op_cond refers to operational condition, sn: sensor
train_df.columns = col_list
test_df.columns = col_list

In [ ]:
train_df.head()

In [ ]:
train_df.info()
#all data are numerical and no missing data

In [ ]:
train_df.describe()


In [ ]:
# visualizing the mean for all the columns 
train_df.mean().plot.bar(figsize=(12,10))

In [ ]:
"Examining the Correlations Between the features using the correlation heatmap"
corr_all=train_df.drop('time',axis=1).corr(method='pearson')# linear correlation between variables for all engines
fig, ax = plt.subplots(figsize=(15,12))
mask = np.zeros_like(corr_all)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr_all,mask=mask,cmap='RdYlGn',linewidths=0.2 )


In [ ]:
def plot_dist(df, engine_num=None):
    '''plot all non trivial measurements and states'''
    cols = df.columns
    n_cols = min(len(cols), 5)
    n_rows = int(np.ceil(len(cols) / n_cols))
    sns.set()
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15,12))
    axes = axes.flatten()
    if engine_num != None: 
        fig.suptitle('distributions for engine #: {}'.format(engine_num))
        df_plot = df.loc[engine_num]
    else: 
        fig.suptitle('distributions for all engines')
        df_plot = df
    for col, ax in zip(cols, axes):
        ax=sns.distplot(df_plot[col], ax=ax, label=col)
        ax.legend(loc=1)
#         labels(col, "p", ax)
    return fig

In [ ]:
fig=plot_dist(train_df)

In [ ]:
"""Feature selection 
i'll be using two approaches to select the features to be used
1- find high correlation column pairs in train_df using a function 
2- Plot the data of the 3 operational settings and 21 sensor measurements to check the trends of the engines from healthy state to failure state."""

In [ ]:
def find_corr_pairs(corr,thrsh):
    """
    find high correlation column pairs in df 
    ======================================
    input: 
    corr - (df)- correlation matrix generated by pandas
    thrsh - (float) threshold value to consider correlation as high so that it is included in the output 
    output: high_corr_pairs - (list) list of tuples of the two-column names and their correlation. corr> thrsh
    """
    high_corr_pairs = []
    # same as input 'corr' but the upper -triangle half of the matrix is zeros ( for convenience only) 
    corr_diag = pd.DataFrame(np.tril(corr.values), columns=corr.columns, index = corr.index)
    # check  the correlation between every pair of columns in the corr and keeps the high ones
    for col_num , col in enumerate(corr_diag):
        col_corr=corr_diag[col].iloc[col_num+1:] # this slicing ensures ignoring self_corr and duplicates due to symmetry
        # bool mask for pairs with high corr with col
        mask_pairs = col_corr.apply(lambda x: abs(x))>thrsh 
        idx_pairs=col_corr[mask_pairs].index
        # create list of high corr pairs
        for idx , corr in zip(idx_pairs,col_corr[mask_pairs].values):
            high_corr_pairs.append((col, idx, corr))
    return high_corr_pairs

In [ ]:
corr_pairs=find_corr_pairs(corr_all,0.8)
for c in corr_pairs:
    print(c)

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows =6, figsize=(24, 20))
ax = ax.ravel()
for i, item in enumerate(col_list[2:]):
  train_df.groupby('engine_num').plot(kind='line', x = "time", y = item, ax=ax[i])
  ax[i].get_legend().remove()
  ax[i].title.set_text(item)
plt.subplots_adjust(top = 0.99, bottom = 0.01, hspace = 0.3, wspace = 0.2)
plt.show()

In [ ]:
""" it's being noticed from above two approches that some feature ain't significant for our anaylsis 
1- based on first approach : sm_4', 'sm_11', 'sm_12','sm_7', 'sm_8', 'sm_13', 'sm_9', 'sm_14' are important 
2- based on second approach : 'op_cond_3', 'sm_1', 'sm_5', 'sm_10', 'sm_16' , 'sm_18', 'sm_19' are not important as they don't change their state in time """

In [ ]:
col_remove = ['op_cond_3', 'sm_1', 'sm_5', 'sm_10', 'sm_16' , 'sm_18', 'sm_19'] 
train_df.drop(columns=col_remove,axis=1,inplace=True)

In [ ]:
train_df.head()

In [ ]:
"this is the maximun cycles that each of the 100 different engines reached to before the data stopped(failure occured), so each max cycle (time) is considered to be the value of RUL of an engine"
train_df.groupby('engine_num')['time'].max()


In [ ]:
"""  each engine develops a fault, which can be seen through sensor readings. the data stops for each engine when a failure has occurred for that particular engine. hence the actual RUL is known based on the length of the data, RUL is simply the number of operational cycles after the last cycle that the engine will continue to operate properly"""

In [ ]:
# Data Labeling - generate column RUL 
rul = pd.DataFrame(train_df.groupby('engine_num')['time'].max()).reset_index()
rul.columns = ['engine_num', 'max']
train_df = train_df.merge(rul, on=['engine_num'], how='left')
train_df['RUL'] = train_df['max'] - train_df['time']
train_df.drop('max', axis=1, inplace=True)
"observed that whenerver the time(cycle) increases for a particular enginer , the RUL is inversely propotional with it as it's decreasing until it reaches 0 (failure)"
train_df.head(192)


In [ ]:
corr = train_df.corr()
print(corr['RUL'].sort_values(ascending=False))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [ ]:
# ploting all the train data by sensors
t = train_df.iloc[0:,2:-1].plot(subplots=True, figsize=(25, 17))


In [ ]:
# inverse the RUL for the plot
train_df['RUL'] = train_df['RUL'] * -1
# plot all engines sensor data
g = sns.PairGrid(data=train_df, x_vars='RUL', y_vars=train_df.iloc[0:,2:-1], hue="engine_num", height=2, aspect=6,)
g = g.map(plt.plot)
g = g.set(xlim=(train_df['RUL'].min(), train_df['RUL'].max()))


In [ ]:
"Prepararing the training and the testing dataset "
def get_train_test(train_df):
  engine_nums = train_df.engine_num.unique()
  np.random.seed(0)
  engine_num_train = np.random.choice(engine_nums, int(len(engine_nums)*0.8), replace=False)
  engine_num_test = np.array(list(set(engine_nums) - set(engine_num_train))) 

  x_train = train_df[train_df.engine_num==engine_num_train[0]]
  y_train = train_df.RUL[train_df.engine_num==engine_num_train[0]]
  for engine_num in engine_num_train[1:]:
     x_train = pd.concat([x_train, train_df[train_df.engine_num==engine_num]])
     y_train = pd.concat([y_train, train_df.RUL[train_df.engine_num==engine_num]])

  x_test = train_df[train_df.engine_num==engine_num_test[0]]
  y_test = train_df.RUL[train_df.engine_num==engine_num_test[0]]
  for engine_num in engine_num_test[1:]:
     x_test = pd.concat([x_test, train_df[train_df.engine_num==engine_num]])
     y_test = pd.concat([y_test, train_df.RUL[train_df.engine_num==engine_num]])

  x_train.drop(['engine_num', 'time', 'RUL'], axis=1, inplace=True)
  x_test.drop(['engine_num', 'time', 'RUL'], axis=1, inplace=True)
  return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test =get_train_test(train_df)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
reg_lin = LinearRegression()
reg_lin.fit(x_train, y_train)
predictions = reg_lin.predict(x_test)
plt.figure()
plt.scatter(y_test, predictions, s=5, alpha=0.15)
plt.xlabel('RUL (true)')
plt.ylabel('RUL')

In [ ]:
def custom_loss(y_true, y_pred, weights=None):
  diff = np.array(y_pred, dtype=np.float) - np.array(y_true, dtype=np.float)
  mask = diff < 0
  diff[mask] = np.exp(-diff[mask]*0.077) - 1
  mask = diff >= 0
  diff[mask] = np.exp(diff[mask]*0.1) - 1
  return diff.sum()

predictions[predictions < 1] = 1
print('MSE', mean_squared_error(y_test, predictions))
print('R2 score', r2_score(y_test, predictions))

In [ ]:
"""
 Reference:
*https://www.mathworks.com/help/predmaint/ug/rul-estimation-using-rul-estimator-models.html
*https://medium.com/@hamalyas_/
*takashi matsushita github
"""